In [1]:
import ftplib
from sqlalchemy import create_engine, text
from sqlalchemy.sql import select
from bs4 import BeautifulSoup
import numpy as np 
from io import BytesIO
import pandas as pd
import time 
from urllib.request import urlopen
import numexpr as ne
import requests

This notebook outlines how to dump the lightcurves and alerts into databases. Let's use SQLAlchemy Core.

First, let's put the lightcurves into the database.

In [2]:
engine = create_engine('sqlite:///foo.db')
conn = engine.connect()

# MOA

In [3]:
# So we don't have to deal with the log10 complaining.
import warnings
warnings.filterwarnings("ignore")

url = "http://www.massey.ac.nz/~iabond/moa/alert2022/alert.php"
response = urlopen(url)
html = response.read()
response.close()
soup = BeautifulSoup(html,"html.parser")
links = soup.find_all('a', href=True)
alert_dirs = []
# Get a list of all the bulge microlensing alerts
for ii, link in enumerate(links):
    if 'BLG' in link.text:
        alert_dirs.append(links[ii]['href'])
        
counter = 0

t0 = time.time()

for nn, alert_dir in enumerate(alert_dirs[0:10]):
    url = "http://www.massey.ac.nz/~iabond/moa/alert2022/" + alert_dir
    response = urlopen(url)
    html = response.read()
    response.close()
    soup = BeautifulSoup(html,"html.parser")

    # Get the magnitude and flux offsets.
    foo = soup.find('b').next_sibling
    moff = foo.split('=')[1].split('-')[0].strip(' ')
    bah = soup.find('sub').next_sibling
    foff = bah.split('+')[1].split(')')[0].strip(' ')

    # Now convert these into floats
    m = ne.evaluate(moff)
    f = ne.evaluate(foff)

    # Now scrape the .dat file into a pandas dataframe.
    url = "https://www.massey.ac.nz/~iabond/moa/alert2022/fetchtxt.php?path=moa/ephot/phot-" + \
            alert_dir.strip('display.php?id=') + ".dat"
    bytes_data = requests.get(url).content
    df = pd.read_csv(BytesIO(bytes_data), 
                     delim_whitespace=True, skiprows=11, skipfooter=1, header=None, engine='python', 
                     names=['hjd', 'delta_flux', 'flux_err', 'foo1', 'foo2', 'foo3', 'foo4', 'foo5'])

    df['mag'] = m - 2.5*np.log10(df['delta_flux'] + f)
    df['mag_err'] = 1.09 * df['flux_err']/(df['delta_flux'] + f)
    df['alert_name'] = 'MB22' + str(nn + 1).zfill(3)  # need to make sure this always works.

    df.dropna(axis='index', how='any', inplace=True)

    cols = ['hjd', 'mag', 'mag_err', 'alert_name']
    df[cols].to_sql(con=engine, schema=None, name="moa", if_exists="append", index=False)
    counter += 1
t1 = time.time()

print('Time to write table: {0:.0f} sec for {1} alerts'.format(t1 - t0, counter))

Time to write table: 22 sec for 10 alerts


# OGLE

In [4]:
ftp = ftplib.FTP("ftp.astrouw.edu.pl")
ftp.login()
ftp.cwd("ogle/ogle4/ews/2019/")

counter = 0

t0 = time.time()

for nn in np.arange(start=1, stop=10, step=1):
    ftp.cwd("blg-" + str(nn).zfill(4))
    
    flo = BytesIO()
    ftp.retrbinary('RETR phot.dat', flo.write)
    flo.seek(0)
    df = pd.read_fwf(flo, header=0, names=['hjd', 'mag', 'mag_err', 'see', 'sky'])
    
    df['alert_name'] = 'OB19' + str(nn + 1).zfill(4) 

    cols = ['hjd', 'mag', 'mag_err', 'alert_name']
    df[cols].to_sql(con=engine, schema=None, name="ogle", if_exists="append", index=False)
    
    ftp.cwd("../")
    
    counter += 1
t1 = time.time()

print('Time to write table: {0:.0f} sec for {1} alerts'.format(t1 - t0, counter))

Time to write table: 18 sec for 9 alerts


# KMTNet

In [5]:
counter = 0

t0 = time.time()

for nn in np.arange(start=1, stop=11, step=1):
    # For KMTNet, get data from all the telescopes?
    url = "https://kmtnet.kasi.re.kr/~ulens/event/2022/view.php?event=KMT-2022-BLG-" + str(nn).zfill(4)
    response = urlopen(url)
    html = response.read()
    response.close()
    soup = BeautifulSoup(html,"html.parser")
    
    links = soup.find_all('a', href=True)

    # Only keep I-band lightcurves. 
    pysis_names = links[3].get_text(separator=',').split(',')[:-2]
    
    for pysis_name in pysis_names:
        url = "https://kmtnet.kasi.re.kr/~ulens/event/2022/data/KB22" + str(nn).zfill(4) + "/pysis/" + pysis_name
        bytes_data = requests.get(url).content
        try:
            df = pd.read_csv(BytesIO(bytes_data), 
                             delim_whitespace=True, skiprows=1, header=None, 
                             names=['hjd', 'Delta_flux', 'flux_err', 'mag', 'mag_err', 'fwhm', 'sky', 'secz'])

            df['alert_name'] = 'KB22' + str(nn + 1).zfill(4) 
            df['lightcurve'] = pysis_name

            cols = ['hjd', 'mag', 'mag_err', 'lightcurve', 'alert_name']
            df[cols].to_sql(con=engine, schema=None, name="kmtnet", if_exists="append", index=False)
            counter += 1
        except:
            print('This doesn\'t exist, skipping.'.format(nn))
            continue
t1 = time.time()

print('Time to write table: {0:.0f} sec for {1} files ({2} alerts)'.format(t1 - t0, counter, nn))

Time to write table: 79 sec for 109 files (10 alerts)


In [6]:
counter = 0

t0 = time.time()

for nn in np.arange(start=1, stop=11, step=1):
    # For KMTNet, get data from all the telescopes?
    url = "https://kmtnet.kasi.re.kr/~ulens/event/2022/view.php?event=KMT-2022-BLG-" + str(nn).zfill(4)
    response = urlopen(url)
    html = response.read()
    response.close()
    soup = BeautifulSoup(html,"html.parser")
    
    links = soup.find_all('a', href=True)

    # Only keep I-band lightcurves. 
    # FIXME: Could probably trim this further down by only taking the one with the most data?
    # But not sure if checking is more expensive than just writing it.
    pysis_names = links[3].get_text(separator=',').split(',')[:-2]
    
    for pysis_name in pysis_names:
        if '_I.pysis' in pysis_name:
            url = "https://kmtnet.kasi.re.kr/~ulens/event/2022/data/KB22" + str(nn+1).zfill(4) + "/pysis/" + pysis_name
            bytes_data = requests.get(url).content
            try:
                df = pd.read_csv(BytesIO(bytes_data), 
                                 delim_whitespace=True, skiprows=1, header=None, 
                                 names=['hjd', 'Delta_flux', 'flux_err', 'mag', 'mag_err', 'fwhm', 'sky', 'secz'])

                df['alert_name'] = 'KB22' + str(nn + 1).zfill(4) 
                df['lightcurve'] = pysis_name
                
                cols = ['hjd', 'mag', 'mag_err', 'lightcurve', 'alert_name']
                df[cols].to_sql(con=engine, schema=None, name="kmtnet", if_exists="append", index=False)
                counter += 1
            except:
                print('This doesn\'t exist, skipping.'.format(nn))
                continue
t1 = time.time()

print('Time to write table: {0:.0f} sec for {1} files ({2} alerts)'.format(t1 - t0, counter, nn))

Time to write table: 45 sec for 56 files (10 alerts)


In [7]:
# s = text('DROP TABLE kmtnet')
# result = conn.execute(s)

# s = text('DROP TABLE ogle')
# result = conn.execute(s)

# s = text('DROP TABLE moa')
# result = conn.execute(s)

In [8]:
# First way to query the table.
# result = engine.execute("SELECT HJD FROM kmtnet").fetchall()

# Second way to query the table.
# s = text('SELECT * FROM kmtnet')
# result = conn.execute(s)
# result.fetchall()

Next, we'll put the alerts into the database. 

Do we want to post all alert values, or just the subset I have here?

# MOA

In [9]:
url = "http://www.massey.ac.nz/~iabond/moa/alert2022/alert.php"
response = urlopen(url)
html = response.read()
response.close()
soup = BeautifulSoup(html,"html.parser")

# Grab columns for tE and Ibase.
tE = soup.find_all('td')[4::8]
Ibase = soup.find_all('td')[6::8]

# Convert them from strings to floats.
tE_list = [float(ne.evaluate(item.get_text())) for item in tE]
Ibase_list = [float(ne.evaluate(item.get_text())) for item in Ibase]

# Now, grab the classification column.
cat = soup.find_all('td')[7::8]
cat_list = [item.get_text() for item in cat]

# Link to the alert page.
alert_url = soup.find_all('td')[0::8]
moa_alert_url = 'http://www.massey.ac.nz/~iabond/moa/alert2022/'
alert_url_list = [moa_alert_url + item.find_all('a', href=True)[0]['href'] for item in alert_url]

# Alert name
nn = len(tE_list)
alert_name = []
for ii in np.arange(nn):
    alert_name.append('MB22' + str(ii+1).zfill(3))

# Put it all into a dataframe.
df = pd.DataFrame(list(zip(alert_name, cat_list, tE_list, Ibase_list, alert_url_list)),
                 columns =['alert_name', 'class', 'tE', 'Ibase', 'alert_url'])

df.to_sql(con=engine, schema=None, name="moa_alerts", if_exists="replace", index=False)

# OGLE 

In [10]:
def ogle_str_to_float(item):
    try:
        return float(ne.evaluate(item.contents[0].replace(u'\n', '')))
    except:
        return
    
# Get alerts using beautiful soup.
url = "https://ogle.astrouw.edu.pl/ogle4/ews/2019/ews.html"
response = urlopen(url)
html = response.read()
response.close()
soup = BeautifulSoup(html,"html.parser")

# Grab columns for tE and Ibase.
tE = soup.find_all('td')[8::15] 
Ibase = soup.find_all('td')[13::15]

# Convert them from strings to floats.
tE_list = [ogle_str_to_float(item) for item in tE]
Ibase_list = [ogle_str_to_float(item) for item in Ibase]
    
# Alert name and page link.
nn = len(tE_list)
alert_name = []
alert_url_list = []
ogle_alert_url = 'https://ogle.astrouw.edu.pl/ogle4/ews/'

for ii in np.arange(nn):
    alert_name.append('OB19' + str(ii+1).zfill(4))
    alert_url_list.append(ogle_alert_url + str(ii+1).zfill(4) + '.html')

# Put it all into a dataframe.
df = pd.DataFrame(list(zip(alert_name, tE_list, Ibase_list, alert_url_list)),
                 columns =['alert_name', 'tE', 'Ibase', 'alert_url'])

df.to_sql(con=engine, schema=None, name="ogle_alerts", if_exists="replace", index=False)

# KMTNet

In [12]:
def kmtnet_str_to_float(item):
    try:
        return float(ne.evaluate(item.get_text().replace(u'\xa0', u'')))
    except:
        return

year = '2022'
url = "https://kmtnet.kasi.re.kr/~ulens/event/" + year + "/"
response = urlopen(url)
html = response.read()
response.close()
soup = BeautifulSoup(html,"html.parser")

if year in ['2022', '2020', '2017', '2016']:
    tE = soup.find_all('td')[7::15][1:]
    Ibase = soup.find_all('td')[10::15][1:]
    cat = soup.find_all('td')[3::15][1:]
elif year in ['2021', '2019', '2018']:
    tE = soup.find_all('td')[8::16][1:]
    Ibase = soup.find_all('td')[11::16][1:]
    cat = soup.find_all('td')[4::16][1:]
else:
    raise Exception('Not a valid year')

tE_list = [kmtnet_str_to_float(item) for item in tE]
Ibase_list = [kmtnet_str_to_float(item) for item in Ibase]
cat_list = [item.get_text().replace(u'\xa0', u'') for item in cat]

# Link to the alert page.
if year in ['2022', '2020', '2017', '2016']:
    alert_url = soup.find_all('td')[0::15][1:]
elif year in ['2021', '2019', '2018']:
    alert_url = soup.find_all('td')[0::16][1:]

else:
    raise Exception('Not a valid year')

kmt_alert_url = 'https://kmtnet.kasi.re.kr/~ulens/event/' + year + '/'
alert_url_list = [kmt_alert_url + item.find_all('a', href=True)[0]['href'] for item in alert_url]

# Alert name
nn = len(tE_list)
alert_name = []
for ii in np.arange(nn):
    alert_name.append('KB22' + str(ii+1).zfill(4))

# Put it all into a dataframe.
df = pd.DataFrame(list(zip(alert_name, cat_list, tE_list, Ibase_list, alert_url_list)),
                 columns =['alert_name', 'class', 'tE', 'Ibase', 'alert_url'])

df.to_sql(con=engine, schema=None, name="kmt_alerts", if_exists="replace", index=False)